## Blending ensemble
Afer several attempt to beat or improve the baseline NBSVM, i found this method of ensemble used by kaggle grandmaster(Netflix winner) [stacking and blending](http://www.chioka.in/stacking-blending-and-stacked-generalization/). the basic summary is that we train with different model.

`In stacking, the combining mechanism is that the output of the classifiers (Level 0 classifiers) will be used as training data for another classifier (Level 1 classifier) to approximate the same target function. Basically, you let the Level 1 classifier to figure out the combining mechanism.`from the post.

[ensemble guide](https://mlwave.com/kaggle-ensembling-guide/). and the output from each of this model is store and squished up to [0,1] . then convert to submission file. the final model train on the extracted feature data from other model `dataset_blend_train` this is obtain from the holdout data like the valdation which is generated for each k-fold, the prediction of this holdout data is store for all the model before the last model, this is then tran on the final model which then blend it.
the result of the final model is then blended with the previous prediction on the `test_data` store and then this final prediction is then squeezed to 0,1.

for instance if i used 5model, if you add up prediction the highest pred can be 5, then it has to be convert to 1.

In [1]:
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

/home/longbridge/tensorflow3/venv3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pandas as pd
import re

In [3]:
test_df  = pd.read_csv("test_clean.csv")
train_df = pd.read_csv("train_clean.csv")

In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [5]:
train_df['Text'] = train_df['Text'].map(lambda com : clean_text(com))
test_df['Text'] = test_df['Text'].map(lambda com : clean_text(com))

In [6]:
labels  = train_df.columns[3:]

In [7]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## Preprocess text
I used tf-idf to preprocess the dataset and use a range from uni to trigram to extract text features,

In [9]:
Text = 'Text'
n = train_df.shape[0]
vec = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [10]:
trn_term_doc = vec.fit_transform(train_df[Text])
test_term_doc = vec.transform(test_df[Text])

In [11]:
np.random.seed(0)  # seed to shuffle the train set

n_folds = 10
verbose = True
shuffle = True

In [12]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

## score
this model give me my first 0.040....

In [13]:
preds = np.zeros((test_term_doc.shape[0],27))
for count,label in enumerate(labels):
    
  
      y = train_df[label].values

      X = trn_term_doc
    #   print(count)
    #   if count > 0:
    #     break

      if shuffle:

        idx = np.random.permutation(y.size)
        X = X[idx]
        y = y[idx]

      skf = list(StratifiedKFold(y, n_folds))

      clfs = [RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
              AdaBoostClassifier(),
              LGBMClassifier(),
              BaggingClassifier(n_estimators=20,n_jobs=-1),
              GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=100)]

      print("Creating train and test sets for blending.")

      dataset_blend_train = np.zeros((X.shape[0],len(clfs)))
      dataset_blend_test = np.zeros((test_term_doc.shape[0],len(clfs)))

      for j,clf in enumerate(clfs):
        print("----",j,clf,"-----")
        dataset_blend_test_j = np.zeros((test_term_doc.shape[0],len(skf)))

        for i,(train,test) in enumerate(skf):

          print("Fold: ",i)
          X_train = X[train]
          y_train = y[train]
          X_test = X[test]
          y_test = y[test]
          clf.fit(X_train,y_train)
          y_submission = clf.predict_proba(X_test)[:,1]
          dataset_blend_train[test,j] = y_submission
          dataset_blend_test_j[:,i] = clf.predict_proba(test_term_doc)[:,1]

        dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)
      print(" ")
      print("Blending.")
      clf = LogisticRegression()
      clf.fit(dataset_blend_train,y)
      y_submission = clf.predict_proba(dataset_blend_test)[:,1]

      print("Linear strecth of predictions to [0,1]")
      y_submission = (y_submission - y_submission.min())/(y_submission.max() - y_submission.min())

      for index in range(0,998):
        if y_submission[index] >=0.5:
          y_submission[index] = 1

        else:
          y_submission[index] = 0

      preds[:,count] = y_submission
      print("------END",label,"-------")

Creating train and test sets for blending.
---- 0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False) -----
Fold:  0
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
---- 1 AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None) -----
Fold:  0
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
---- 2 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.00

KeyboardInterrupt: 

## score
also give score 0.040 and 0.0401

In [ ]:
preds = np.zeros((test_term_doc.shape[0],len(labels)))
for count,label in enumerate(labels):
    print("label: ",label)
    y = train_df[label].values
    r = np.log(pr(1,y) / pr(0,y))
    X = trn_term_doc
   
    if shuffle:
        idx = np.random.permutation(y.size)
        X = X[idx]
        y = y[idx]
        
    skf = list(StratifiedKFold(y, n_folds))

    clfs = [LinearSVC(),
          LogisticRegression(dual=True),
          MultinomialNB(),
          LGBMClassifier()]

    print("creating train and test sets for blending.")

    dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
    dataset_blend_test = np.zeros((test_term_doc.shape[0], len(clfs)))

    for j, clf in enumerate(clfs):
        print("--",j, clf,"--")
        dataset_blend_test_j = np.zeros((test_term_doc.shape[0], len(skf)))
        for i, (train, test) in enumerate(skf):
            print("Fold", i)
            X_train = X[train]
            y_train = y[train]
            X_test = X[test]
            y_test = y[test]
            if j==0:
                clf.fit(X_train.multiply(r),y_train)
            else:

                clf.fit(X_train,y_train)
            if j==0 or j==1:
                y_submission = clf.predict(X_test.multiply(r))
                dataset_blend_train[test,j] = y_submission
                dataset_blend_test_j[:,i] = clf.predict(test_term_doc.multiply(r))
            else:

                y_submission = clf.predict_proba(X_test)[:,1]
                dataset_blend_train[test,j] = y_submission
                dataset_blend_test_j[:,i] = clf.predict_proba(test_term_doc)[:,1]
            dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
        

    print()
    print("Blending.")
    clf = LinearSVC()
    clf.fit(dataset_blend_train, y)
    y_submission = clf.predict(dataset_blend_test)

    print("Linear stretch of predictions to [0,1]")
    y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
    
    for index in range(0,998):
        if y_submission[index] >=0.5:
              y_submission[index] = 1

        else:
              y_submission[index] = 0
      
    preds[:,count] = y_submission
    print("------END",label,"-------")